Utilizando o Python para criar um script de carregamento para a "Vendas"

In [1]:
import pandas as pd

Mostrando o conteudo do CSV (Apenas os dois primeiros)

In [2]:
df = pd.read_csv('fato_vendas.csv')
df.head(2)

,id_venda,data_venda,id_cliente,id_produto,quantidade
0,1,2024-05-01,97,46,407
1,2,2024-06-13,34,16,62


Visualizando os metadados do nosso arquivo, para checagem de tipo de dados etc

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_venda    1000 non-null   int64 
 1   data_venda  1000 non-null   object
 2   id_cliente  1000 non-null   int64 
 3   id_produto  1000 non-null   int64 
 4   quantidade  1000 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 39.2+ KB


Primeiro modo, inserção manual, apenas cria um script de inserção.

In [4]:
with open('inserts_vendas.sql', 'w', encoding='utf-8') as f:
    for _, row in df.iterrows():
        
        insert_sql = (
            "INSERT INTO vendas (id_venda, data_venda, id_cliente, id_produto, quantidade) "
            "VALUES ({}, {}, {}, {}, {});\n".format(row['id_venda'], row['data_venda'], row['id_cliente'], row['id_produto'], row['quantidade'])
        )
        f.write(insert_sql)

print("Arquivo 'inserts_vendas.sql' gerado com sucesso!")


Arquivo 'inserts_vendas.sql' gerado com sucesso!


Segundo modo, inserção direta no banco de dados, executa os comandos de insert diretamente.

In [ ]:
# Primeiramente, vou utilizar uma biblioteca para conectar ao MySQL e inserir os dados do DataFrame.

from decimal import Decimal
import mysql.connector

# Função para inserir dados no MySQL, definindo o nome da tabela como 'vendas' e a conexão com o banco de dados.

def insert_data_to_mysql(df, table_name='vendas'):
    connection_params = {
        'host': 'localhost',
        'user': 'root',
        'password': 'pw4dummy',
        'database': 'prod_nsk'
    }
    try:
        # Conectar ao banco de dados MySQL
        conn = mysql.connector.connect(**connection_params)
        # Criar um cursor para executar comandos SQL
        cursor = conn.cursor()

        # SQL para inserir dados na tabela
        #
        sql = f"""
        INSERT INTO {table_name}
        (id_venda, data_venda, id_cliente, id_produto, quantidade)
        VALUES (%s, %s, %s, %s, %s)
        """
        count = 0
        for _, row in df.iterrows():
            values = (
                int(row['id_venda']),
                row['data_venda'],
                int(row['id_cliente']),
                int(row['id_produto']),
                int(row['quantidade'])
            )
            cursor.execute(sql, values)
            count += 1

        conn.commit()
        #mensagem de sucesso
        print(f"{count} registros inseridos com sucesso na tabela {table_name}.")

    except mysql.connector.Error as err:
        # Mensagem de erro
        print(f"Erro MySQL: {err}")

    # Fechar o cursor e a conexão
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Executar inserção
insert_data_to_mysql(df)

1000 registros inseridos com sucesso na tabela vendas.
